In [ ]:
from selenium.webdriver.support.ui import Select, WebDriverWait
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException, NoSuchWindowException
from selenium.webdriver.support import expected_conditions as EC
import re
from tqdm import tqdm  # 引入 tqdm 用於顯示進度條

URL = 'https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E7%86%B1%E9%96%80%E6%8E%92%E8%A1%8C&INDUSTRY_CAT=%E6%88%90%E4%BA%A4%E5%BC%B5%E6%95%B8+%28%E9%AB%98%E2%86%92%E4%BD%8E%29%40%40%E6%88%90%E4%BA%A4%E5%BC%B5%E6%95%B8%40%40%E7%94%B1%E9%AB%98%E2%86%92%E4%BD%8E'

num=300 # 篩選數量 最多300
K_max=20 # 篩選 K 最大
D_max=50 # 篩選 D 最大

# 创建 Edge WebDriver 對象
driver = webdriver.Edge()

# 打開網頁
driver.get(URL)

# 篩選 KD
# select_element = driver.find_element(By.ID, 'selSHEET')
# 網頁更新為動態，等待數秒後執行
select_element = WebDriverWait(driver, 10).until( EC.presence_of_element_located((By.ID, "selSHEET")) )
select = Select(select_element)
select.select_by_value('KD指標')

volume_num = 0
i_num = np.array([], dtype=int)

# 第一個進度條：篩選 K 和 D 值
for volume_num in tqdm(range(num), desc="篩選 K 和 D", ncols=100):
    try:
        # 等待 K 和 D 元素加载完成
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="row{volume_num}"]/td[9]/nobr/a')))
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="row{volume_num}"]/td[10]/nobr/a')))

        try:
            K_element = driver.find_element(By.XPATH, f'//*[@id="row{volume_num}"]/td[9]/nobr/a')
            K_text = K_element.text
            D_element = driver.find_element(By.XPATH, f'//*[@id="row{volume_num}"]/td[10]/nobr/a')
            D_text = D_element.text
            
            # 從文本中提取 K 值和 D 值的前兩個數字
            K_match = re.match(r'\d+', K_text)
            D_match = re.match(r'\d+', D_text)
            
            if K_match and D_match:
                K_value = K_match.group()[:2] if K_match.group() else None
                D_value = D_match.group()[:2] if D_match.group() else None
                if K_value is not None and D_value is not None:
                    if int(K_value) < K_max and int(D_value) < D_max:
                        i_num = np.append(i_num, volume_num)
        except StaleElementReferenceException:
            continue
    except (TimeoutException, NoSuchElementException):
        continue
    except NoSuchWindowException:
        break

print("篩選出的股票名稱：", i_num)


# 查看 PER 是否存在
select_element = driver.find_element(By.ID, 'selSHEET')
select = Select(select_element)
select.select_by_value('交易狀況')

stock_names_lessETP = np.array([], dtype=int)

# 第二個進度條：查看 PER 是否存在
for i, volume_num in tqdm(enumerate(i_num, start=1), desc="查看 PER", total=len(i_num), ncols=100):
    try:
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="row{volume_num}"]/td[14]/nobr')))
    except TimeoutException:
        pass
    try:
        stock_PER = driver.find_elements(By.XPATH, f'//*[@id="row{volume_num}"]/td[15]/nobr')
        if stock_PER:
            stock_names_lessETP = np.append(stock_names_lessETP, volume_num)
    except TimeoutException:
        continue

print("篩選出的股票號：", stock_names_lessETP)

# 查看 毛利率 是否存在
select_element = driver.find_element(By.ID, 'selSHEET')
select = Select(select_element)
select.select_by_value('季獲利能力')

stock_names_GP = np.array([])

# 第三個進度條：查看毛利率
for i, volume_num in tqdm(enumerate(stock_names_lessETP, start=1), desc="查看毛利率", total=len(stock_names_lessETP), ncols=100):
    try:
        WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="row{volume_num}"]/td[2]/nobr')))
        stock_name = driver.find_element(By.XPATH, f'//*[@id="hrow{volume_num}"]/td[3]/nobr/a')
        stock_GP = driver.find_elements(By.XPATH, f'//*[@id="row{volume_num}"]/td[10]/nobr')

        if stock_GP:
            stock_names_GP = np.append(stock_names_GP, stock_name.text)
    except (StaleElementReferenceException, TimeoutException):
        continue

print("篩選出的股票數量：", len(stock_names_GP))
print("篩選出的股票名稱：", stock_names_GP)
